# Step 3: Model Operationalization & Deployment

In the previous script, you learned how to save lstm trained models to files. You also learned that model weights are easily stored using  HDF5 format and that the network structure can be saved in JSON. In this script, you will learn how to load your models up, operationalize and use them to make future predictions.

In [1]:
import keras
# import the libraries
import os
import pandas as pd
import numpy as np
import urllib
import json
import shutil
from keras.models import model_from_json

from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
#from keras.layers import Dense, Dropout, LSTM, Activation gdt

import h5py
from sklearn import datasets

# For creating the deployment schema file
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.operationalization','true')

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

Using TensorFlow backend.


## Model storage 

We will stor the model in an Azure Blob Storage Container for easy retreival to your deployment platform. 
Instructions for setting up your Azure Storage account are available within this link (https://docs.microsoft.com/en-us/azure/storage/blobs/storage-python-how-to-use-blob-storage). You will need to copy your account name and account key from the _Access Keys_ area in the portal into the following code block. These credentials will be reused in all four Jupyter notebooks.

We will handle creating the containers and writing the data to these containers for each notebook. Further instructions for using Azure Blob storage with AML Workbench are available (https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md).

You will need to enter the **ACCOUNT_NAME** as well as the **ACCOUNT_KEY** in order to access Azure Blob storage account you have created. This notebook will create and store all the resulting data files in a blob container under this account.


In [2]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<ACCOUNT_NAME>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<ACCOUNT_KEY>"

#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
MODEL_CONTAINER = "pmlstmmodel" 
# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

In [3]:
# We will store each of these data sets in a local persistance folder
SHARE_ROOT = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

# These file names detail the data files. 
TEST_DATA = 'PM_test_files.pkl'

# We'll serialize the model in json format
LSTM_MODEL = 'modellstm.json'

# and store the weights in h5
MODEL_WEIGHTS = 'modellstm.h5'

## Load the test data frame

We haev previously stored the test data frame in the local persistance directory indicated by the _SHARE_ROOT_ variable. We'll use this data frame to test the model deployment and build the model schema to describe the deployment function calls.

In [4]:
test_df = pd.read_pickle(SHARE_ROOT + TEST_DATA)
test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138,0,0
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137,0,0
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136,0,0
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135,0,0
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134,0,0
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133,0,0


In [5]:
with open(SHARE_ROOT + LSTM_MODEL, 'r') as json_file:
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    print("json file read from shared folder")

json file read from shared folder


In [6]:
with open(SHARE_ROOT + MODEL_WEIGHTS, 'r') as model:
    loaded_model.load_weights(os.path.join(SHARE_ROOT, MODEL_WEIGHTS))
    print("model weights read from shared folder")

model weights read from shared folder


### Test init() and run() functions to read from the working directory

In [7]:
# pick a large window size of 50 cycles
sequence_length = 50

In [8]:
def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    with open(SHARE_ROOT + LSTM_MODEL, 'r') as json_file:
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    with open(SHARE_ROOT + MODEL_WEIGHTS, 'r') as model:
        print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")


In [9]:
# pick the feature columns 
# pick the feature columns 
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
key_cols = ['id', 'cycle']
label_cols = ['label1', 'label2', 'RUL']

input_features = test_df.columns.values.tolist()
sensor_cols = [x for x in input_features if x not in set(key_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols.extend(sensor_cols)

print(sequence_cols)

seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

['setting1', 'setting2', 'setting3', 'cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']


(93, 50, 25)

In [10]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [11]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(93, 1)

In [12]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(93, 50, 25)
(93, 1)


In [13]:
# test metrics
prediction = loaded_model.predict_proba(seq_array_test_last, verbose=2)
prediction

array([[3.9441944e-03],
       [1.5420046e-03],
       [2.2944731e-03],
       [1.6342408e-03],
       [1.0711822e-03],
       [1.8137152e-03],
       [1.5652634e-03],
       [1.4222763e-03],
       [1.7837969e-03],
       [1.1179396e-03],
       [1.3857153e-03],
       [9.7430602e-04],
       [1.0974925e-03],
       [4.5014494e-03],
       [7.2531439e-02],
       [1.2426327e-03],
       [5.1758206e-01],
       [1.3090482e-03],
       [8.1619818e-04],
       [7.8406298e-01],
       [9.3027839e-04],
       [2.0040979e-03],
       [1.3282114e-03],
       [1.5891819e-03],
       [1.8134536e-03],
       [9.7921205e-01],
       [4.2090137e-03],
       [1.4412153e-03],
       [9.6478546e-01],
       [9.7781885e-01],
       [5.7088947e-01],
       [7.6015361e-02],
       [4.7884155e-03],
       [2.2282261e-01],
       [2.7605569e-01],
       [9.3500495e-01],
       [2.4655883e-03],
       [1.6047072e-03],
       [2.9705903e-03],
       [1.7792160e-02],
       [1.8148939e-03],
       [1.460879

In [14]:
def run(score_df): 
    global clf2, inputs_dc, prediction_dc

    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']
    label_cols = ['label1', 'label2', 'RUL']

    input_features = test_df.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_df[score_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in score_df['id'].unique() if len(score_df[score_df['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

In [15]:
init()

Loaded model


In [16]:
run(test_df)

array([[3.9441944e-03],
       [1.5420046e-03],
       [2.2944731e-03],
       [1.6342408e-03],
       [1.0711822e-03],
       [1.8137152e-03],
       [1.5652634e-03],
       [1.4222763e-03],
       [1.7837969e-03],
       [1.1179396e-03],
       [1.3857153e-03],
       [9.7430602e-04],
       [1.0974925e-03],
       [4.5014494e-03],
       [7.2531439e-02],
       [1.2426327e-03],
       [5.1758206e-01],
       [1.3090482e-03],
       [8.1619818e-04],
       [7.8406298e-01],
       [9.3027839e-04],
       [2.0040979e-03],
       [1.3282114e-03],
       [1.5891819e-03],
       [1.8134536e-03],
       [9.7921205e-01],
       [4.2090137e-03],
       [1.4412153e-03],
       [9.6478546e-01],
       [9.7781885e-01],
       [5.7088947e-01],
       [7.6015361e-02],
       [4.7884155e-03],
       [2.2282261e-01],
       [2.7605569e-01],
       [9.3500495e-01],
       [2.4655883e-03],
       [1.6047072e-03],
       [2.9705903e-03],
       [1.7792160e-02],
       [1.8148939e-03],
       [1.460879

## Persist model assets

Next we persist the assets we have created to disk for use in operationalization.

In [17]:
# define the input data frame
inputs = {"score_df": SampleDefinition(DataTypes.PANDAS, 
                                       test_df[sequence_cols])}

json_schema = generate_schema(run_func=run, inputs=inputs, filepath='service_schema.json')

# save the schema file for deployment
out = json.dumps(json_schema)
with open(SHARE_ROOT + 'service_schema.json', 'w') as f:
    f.write(out)
    

In [18]:
%%writefile {SHARE_ROOT}/lstmscore.py

# import the libraries
import keras
import json
import shutil

def init():
    # read in the model file
    from keras.models import model_from_json

    # load json and create model
    json_file = open('modellstm.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("modellstm.h5")
    print("Loaded model")
    
    #inputs_dc = ModelDataCollector("modellstm.h5", identifier="inputs")
    #print(inputs_dc)
    #prediction_dc = ModelDataCollector("modellstm.h5", identifier="prediction")
    
    
def run(score_df): 
    global clf2, inputs_dc, prediction_dc

    # Create the sequences
    sequence_length = 50
    sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
    key_cols = ['id', 'cycle']
    label_cols = ['label1', 'label2', 'RUL']

    input_features = test_df.columns.values.tolist()
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_df[score_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in score_df['id'].unique() if len(score_df[score_df['id']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = loaded_model.predict_proba(seq_array)
        #print(prediction)
        #inputs_dc.collect(seq_array)
        #prediction_dc.collect(prediction)
        return (prediction)
    except Exception as e:
        return(str(e))

Overwriting /azureml-share//lstmscore.py


and generate the schema file

## Packaging

To move the model artifacts around, we'll zip them into one file. We can then retreive this file from the persistance shared folder on your DSVM.

https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files



In [19]:
# Compress the operationalization assets for easy blob storage transfer
# We can remove the persisted data files.
# !rm {SHARE_ROOT}/PM*.pkl

MODEL_O16N = shutil.make_archive('LSTM_o16n', 'zip', SHARE_ROOT)

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
az_blob_service.create_container(MODEL_CONTAINER,
                                 fail_on_exist=False, 
                                 public_access=PublicAccess.Container)

# Transfer the compressed operationalization assets into the blob container.
az_blob_service.create_blob_from_path(MODEL_CONTAINER, "LSTM_o16n.zip", str(MODEL_O16N)) 

## Deployment

Once the assets are stored, we can download them into a local compute context for operationalization on an Azure web service.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

Once downloaded, unzip the file into the directory of your choosing. The zip file contains three deployment assets:

- the `lstmscore.py` file
- a `lstm.model` directory
- the `modellstm.json` file



## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `lstmmodelmanagement`. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name lstmmodelmanagement`


## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with the existing model management context first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`


## Deploy your web service 

Once the environment is setup, we'll deploy the web service from the CLI.

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (`lstmscore.py`, `modellstm.json` and `lstm.model`). If your kernel has run locally, the assets will be in the `os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']`. 

On windows this points to:

`
cd C:\Users\<username>\.azureml\share\<team account>\<Project Name>
`

on linux variants this points to:

`
cd ~\.azureml\share\<team account>\<Project Name>
`


The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

`
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
`

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice `amlworkbenchpdmwebservice`. This `SERVICE_ID` must be all lowercase, with no spaces:

`
az ml service create realtime -f lstmscore.py -r spark-py -m lstm.model -s modellstm.json --cpu 0.1 -n amlworkbenchpdmwebservice
`

This command will take some time to execute. 

Once complete, the command returns sample usage commands to test the service for both PowerShell and the cmd prompt. We can execute these commands from the command line as well.

# Deployment

Once the assets are stored, we can download them into a deployment compute context for operationalization on an Azure web service. For this scenario, we will deploy this on our local docker container context.

We demonstrate how to setup this web service this through a CLI window opened in the AML Workbench application. 

## Download the model

To download the model we've saved, follow these instructions on a local computer.

- the `lstmscore.py` file which contains functionst to do the model scoring
- the `modellstm.json` model definition file
- the `modellstm.h5` model weights file
- the `service_schema.json` which defines the input data schema


## Create a model management endpoint 

Create a modelmanagement under your account. We will call this `pdmmodelmanagement`. The remaining defaults are acceptable.

`az ml account modelmanagement create --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

If you get a `ResourceGroupNotFound` error, you may need to set the correct subscription. This is typically only an issue if your Azure login connects to multiple subscritpions. 

`az account set -s '<subscription name>'`

You can find the `subscription name` or `subscription id` through the (https://portal.azure.com) under the resource group you'd like to use.

## Check environment settings

Show what environment is currently active:

`az ml env show`

If nothing is set, we setup the environment with the existing model management context first: 

` az ml env setup --location <ACCOUNT_REGION> --resource-group <RESOURCE_GROUP> --name pdmmodelmanagement`

using the same `<ACCOUNT_REGION>` and `<RESOURCE_GROUP>` in the previous section. Then set the current environment:

`az ml env set --resource-group <RESOURCE_GROUP> --cluster-name pdmmodelmanagement`

Check that the environment is now set:

`az ml env show`


## Install docker to compute target

Once the model management environment is setup, we'll deploy the web service from the CLI to a local docker container for this demonstration. This assumes you have docker installed localy (https://www.docker.com/get-docker).

Once docker is installed and running, you will need to prepare the local docker container, just as we didi the remote container.

`az ml experiment prepare -c docker`

Now deploy the solution to this container.


## Deploy a web service 

These commands assume the current directory contains the webservice assets we created in throughout the notebooks in this scenario (`lstmscore.py`, `modellstm.json`, `modellstm.h5` and `service_schema.json`). Change to the directory where the zip file was unpacked. 

The command to create a web service (`<SERVICE_ID>`) with these operationalization assets in the current directory is:

`
az ml service create realtime -f <filename> -r <TARGET_RUNTIME> -m <MODEL_FILE> -s <SCHEMA_FILE> -n <SERVICE_ID> --cpu 0.1
`

The default cluster has only 2 nodes with 2 cores each. Some cores are taken for system components. AMLWorkbench asks for 1 core per service. To deploy multiple services into this cluster, we specify the cpu requirement in the service create command as (--cpu 0.1) to request 10% of a core. 

For this example, we will call our webservice `lstmwebservice`. This `SERVICE_ID` must be all lowercase, with no spaces:

`
az ml service create realtime -f lstmscore.py -r python -m modellstm.json -m modellstm.h5 -s service_schema.json --cpu 0.1 -n lstmwebservice
`

This command will take some time to execute. 

## Test your deployment.

Once complete, the `az ml service create` command returns sample usage commands to test the service for both PowerShell and the cmd prompt. We can test this deployment by executing these commands from the command line. For our example:

```
 az ml service run realtime -i amlworkbenchpdmwebservice --% -d "{\"input_df\": [{\"rotate_rollingmean_36\": 450.0384342542265, \"rotate_rollingstd_36\": 0.0, \"volt_rollingmean_24\": 166.69782028530955, \"volt_rollingmean_36\": 166.5072079613422, \"comp1sum\": 504.0, \"comp2sum\": 564.0, \"error3sum_rollingmean_24\": 0.0, \"vibration_rollingmean_24\": 40.302192663278625, \"pressure_rollingstd_24\": 0.0, \"vibration_rollingstd_12\": 0.0, \"pressure_rollingstd_12\": 0.0, \"rotate_rollingmean_12\": 445.7130438343768, \"volt_rollingstd_24\": 0.0, \"comp3sum\": 444.0, \"error1sum_rollingmean_24\": 0.0, \"vibration_rollingstd_36\": 0.0, \"rotate_rollingstd_24\": 0.0, \"volt_rollingstd_36\": 0.0, \"rotate_rollingmean_24\": 444.92430808877185, \"error2sum_rollingmean_24\": 0.0, \"pressure_rollingstd_36\": 0.0, \"comp4sum\": 399.0, \"machineID\": 27, \"pressure_rollingmean_24\": 100.42784289855126, \"volt_rollingmean_12\": 162.37456132546583, \"error4sum_rollingmean_24\": 0.0, \"pressure_rollingmean_12\": 103.46853199581041, \"age\": 9, \"pressure_rollingmean_36\": 99.1626730910439, \"volt_rollingstd_12\": 0.0, \"rotate_rollingstd_12\": 0.0, \"vibration_rollingmean_36\": 39.86004229336383, \"vibration_rollingstd_24\": 0.0, \"error5sum_rollingmean_24\": 0.0, \"vibration_rollingmean_12\": 39.69610732198209}, {\"rotate_rollingmean_36\": 452.58602482190344, \"rotate_rollingstd_36\": 1.3063227195446807, \"volt_rollingmean_24\": 168.8315798036505, \"volt_rollingmean_36\": 166.8633264221902, \"comp1sum\": 504.0, \"comp2sum\": 564.0, \"error3sum_rollingmean_24\": 0.0, \"vibration_rollingmean_24\": 39.8762193116053, \"pressure_rollingstd_24\": 0.5506261833397947, \"vibration_rollingstd_12\": 0.5581845837178677, \"pressure_rollingstd_12\": 1.3059590035299573, \"rotate_rollingmean_12\": 448.82482383859184, \"volt_rollingstd_24\": 1.1327450423992658, \"comp3sum\": 444.0, \"error1sum_rollingmean_24\": 0.0, \"vibration_rollingstd_36\": 0.12802019423837702, \"rotate_rollingstd_24\": 6.2252625510326345, \"volt_rollingstd_36\": 1.2113288898088435, \"rotate_rollingmean_24\": 455.68853459771736, \"error2sum_rollingmean_24\": 0.0, \"pressure_rollingstd_36\": 0.360813923769749, \"comp4sum\": 399.0, \"machineID\": 27, \"pressure_rollingmean_24\": 98.84197839575184, \"volt_rollingmean_12\": 169.6342364499553, \"error4sum_rollingmean_24\": 0.0, \"pressure_rollingmean_12\": 100.13428527324218, \"age\": 9, \"pressure_rollingmean_36\": 99.18126302139088, \"volt_rollingstd_12\": 1.7162303092954838, \"rotate_rollingstd_12\": 7.358009183124642, \"vibration_rollingmean_36\": 39.83194043387068, \"vibration_rollingstd_24\": 0.26866456414969686, \"error5sum_rollingmean_24\": 0.0, \"vibration_rollingmean_12\": 40.534215611846555}, {\"rotate_rollingmean_36\": 452.6366978657443, \"rotate_rollingstd_36\": 0.726203655443797, \"volt_rollingmean_24\": 165.47787140830766, \"volt_rollingmean_36\": 164.9839282666808, \"comp1sum\": 503.0, \"comp2sum\": 563.0, \"error3sum_rollingmean_24\": 0.0, \"vibration_rollingmean_24\": 39.48080284488274, \"pressure_rollingstd_24\": 0.43573594568766316, \"vibration_rollingstd_12\": 0.33150005427630586, \"pressure_rollingstd_12\": 0.30398746497620055, \"rotate_rollingmean_12\": 462.5522453568429, \"volt_rollingstd_24\": 1.388783538126311, \"comp3sum\": 443.0, \"error1sum_rollingmean_24\": 0.0, \"vibration_rollingstd_36\": 0.06733738203927228, \"rotate_rollingstd_24\": 2.2615583783043336, \"volt_rollingstd_36\": 0.4066137169118576, \"rotate_rollingmean_24\": 454.4666253135592, \"error2sum_rollingmean_24\": 0.0, \"pressure_rollingstd_36\": 0.40800640702349306, \"comp4sum\": 398.0, \"machineID\": 27, \"pressure_rollingmean_24\": 98.70475189546528, \"volt_rollingmean_12\": 168.0289231573457, \"error4sum_rollingmean_24\": 0.0, \"pressure_rollingmean_12\": 97.5496715182615, \"age\": 9, \"pressure_rollingmean_36\": 99.92595364177775, \"volt_rollingstd_12\": 1.9026812928919759, \"rotate_rollingstd_12\": 12.545522310840685, \"vibration_rollingmean_36\": 39.16084871098736, \"vibration_rollingstd_24\": 0.2757178837764945, \"error5sum_rollingmean_24\": 0.0, \"vibration_rollingmean_12\": 39.21822301136402}]}"
```
This submits 3 records to the model through the web service, and returns predictioned output labels for each of the three rows:

```
"0.0,0.0,0.0"
```

Indicating that these records are predicted to be healthy with in the requested 7 day time window.


We can view additional service usage information with the following command. 

```az ml service usage realtime -i amlworkbenchpdmwebservice```

Which indicates how the service is currently deployed:

```
Scoring URL:
    http://127.0.0.1:32770/score

Headers:
    Content-Type: application/json

Swagger URL:
    http://127.0.0.1:32770/swagger.json

Sample CLI command:
...
```

# Conclusion

Working through all of these notebooks, we have completed:

 * Data aquisition in `Code/1_data_aquisition.ipynb` notebook.
 * Time series feature engineering and failure labeling to predict component failures within a 7 day window in the `Code/2_feature_engineering.ipynb` notebook.
 * Model building and evaluation in the `Code/3_model_building.ipynb` notebook.
 * Deployment asset generation and model deployment in the `Code/4_operationalization.ipynb` notebook.
    
